# ACO Hyperparameter Tuning for Graph Coloring Problem

This notebook performs hyperparameter tuning for Ant Colony Optimization (ACO) algorithm applied to the Graph Coloring Problem using Optuna.

## 1. Environment Setup

In [1]:
import sys
import os

# Check if running in Google Colab environment
IS_COLAB = 'google.colab' in sys.modules

print(f"Running in Google Colab: {IS_COLAB}")

if IS_COLAB:
    print("Colab environment detected. Will mount Google Drive.")
    # Mount Google Drive if running in Colab
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted successfully at /content/drive")
else:
    print("Local environment detected. Using local paths.")

Running in Google Colab: False
Local environment detected. Using local paths.


In [2]:
# Configure paths for data, studies, results, and figures based on the execution environment.
from pathlib import Path

# Configure base paths based on environment
if IS_COLAB:
    # Update this path to match your Google Drive structure
    BASE_PATH = Path('/content/drive/MyDrive/meta_graph_coloring_antcol/assignemnt3')
    CODE_PATH = BASE_PATH / 'code'
    # Add code path to system path for imports
    sys.path.insert(0, str(CODE_PATH))
else:
    # Local environment paths
    BASE_PATH = Path('/meta/graph_coloring_antcol/assignemnt3')
    CODE_PATH = BASE_PATH / 'code'

# Define data root path (contains tiny_dataset and main_dataset)
DATA_ROOT = BASE_PATH / 'data'

# Verify paths exist
if not BASE_PATH.exists():
    raise FileNotFoundError(f"Base path does not exist: {BASE_PATH}")
if not DATA_ROOT.exists():
    raise FileNotFoundError(f"Data root does not exist: {DATA_ROOT}")

print(f"Base Path: {BASE_PATH}")
print(f"Code Path: {CODE_PATH}")
print(f"Data Root: {DATA_ROOT}")
print(f"\nPath verification: OK")

Base Path: /meta/graph_coloring_antcol/assignemnt3
Code Path: /meta/graph_coloring_antcol/assignemnt3/code
Data Root: /meta/graph_coloring_antcol/assignemnt3/data

Path verification: OK


In [3]:
# Install required packages if running in Colab
if IS_COLAB:
    print("Installing required packages...")
    !pip install -q networkx==3.2.1 matplotlib==3.8.2 pandas==2.1.4 numpy==1.26.2 optuna==3.5.0 scikit-learn==1.4.0 scipy==1.11.4
    print("Packages installed successfully!")

In [4]:
# Import Required Libraries
from pathlib import Path
import multiprocessing
from datetime import datetime
from IPython.display import Image, display, Markdown, HTML

# Import project modules
from dataloader import GraphDataLoader
from optuna_tuner import OptunaACOTuner
from aco_gpc import ACOGraphColoring
from objective_function import aco_objective_function

print("All libraries imported successfully!")

All libraries imported successfully!


## 2. Study Configuration

Define dataset selection, study name, and hyperparameter search space.

In [5]:
# Dataset selection: 'tiny_dataset' for quick testing, 'main_dataset' for full experiments
DATASET_NAME = 'limited_dataset'  # Change to 'main_dataset' for full tuning

# Study continuation mode
# Option 1: Continue existing study - set CONTINUE_STUDY to the study name
# Example: CONTINUE_STUDY = 'aco_study_tiny_dataset_20251129_172927'
# Option 2: Start new study - set CONTINUE_STUDY = None
CONTINUE_STUDY = None  # Set to study name to continue, or None for new study

# Study name (used for new studies or when CONTINUE_STUDY is None)
if CONTINUE_STUDY:
    STUDY_NAME = CONTINUE_STUDY
    print(f"Continuing existing study: {STUDY_NAME}")
else:
    STUDY_NAME = f'aco_study_{DATASET_NAME}_{datetime.now().strftime("%Y%m%d_%H%M%S")}'
    print(f"Creating new study: {STUDY_NAME}")

# Number of Optuna trials for hyperparameter tuning
N_TRIALS = 40

# Number of random exploration trials before optimization starts
N_STARTUP_TRIALS = 4

# ACO verbose setting 
ACO_VERBOSE = False   # Set to True to see detailed ACO progress

# Hyperparameter search space configuration (parameters to optimize)
PARAM_CONFIG = {
    'iterations': {
        'type': 'int',
        'low': 200,
        'high': 500,
    },
    'alpha': {
        'type': 'float',
        'low': 0.5,
        'high': 3.0,
    },
    'beta': {
        'type': 'float',
        'low': 1.0,
        'high': 6.0,
    },
    'rho': {
        'type': 'float',
        'low': 0.001,
        'high': 0.5,
    },
    'ant_count': {
        'type': 'int',
        'low': 10,
        'high': 100,
    },
    'Q': {
        'type': 'float',
        'low': 0.1,
        'high': 10.0,
    },
    'patience': {
        'type': 'float',
        'low': 0.5,
        'high': 0.7,
    }
}

print(f"Configuration:")
print(f"  Dataset: {DATASET_NAME}")
print(f"  Study Name: {STUDY_NAME}")
print(f"  Optuna Trials: {N_TRIALS}")
print(f"  Random Exploration Trials: {N_STARTUP_TRIALS} (before optimization)")
print(f"\nHyperparameters to Optimize:")
for param_name, param_spec in PARAM_CONFIG.items():
    print(f"  {param_name}: [{param_spec['low']}, {param_spec['high']}] ({param_spec['type']})")

Creating new study: aco_study_limited_dataset_20251130_195639
Configuration:
  Dataset: limited_dataset
  Study Name: aco_study_limited_dataset_20251130_195639
  Optuna Trials: 40
  Random Exploration Trials: 4 (before optimization)

Hyperparameters to Optimize:
  iterations: [200, 500] (int)
  alpha: [0.5, 3.0] (float)
  beta: [1.0, 6.0] (float)
  rho: [0.001, 0.5] (float)
  ant_count: [10, 100] (int)
  Q: [0.1, 10.0] (float)
  patience: [0.5, 0.7] (float)


In [6]:
# Initialize the Optuna tuner
tuner = OptunaACOTuner(
    study_name=STUDY_NAME,
    data_root=str(DATA_ROOT),
    direction='minimize',  # We want to minimize the number of colors used
    n_startup_trials=N_STARTUP_TRIALS  # Number of random trials before optimization
)

print(f"Tuner initialized with study: {STUDY_NAME}")
print(f"Data root: {DATA_ROOT}")

# Load tuning dataset once (before optimization)
print("\nLoading tuning dataset...")
data_loader = GraphDataLoader(str(DATA_ROOT), DATASET_NAME)
tuning_graphs, tabu_best_values = data_loader.load_tuning_dataset()
print(f"Loaded {len(tuning_graphs)} graphs for tuning\n")

# Wrapper function to pass tuning graphs and tabu best values to objective function
def objective_wrapper(trial, params, **kwargs):
    return aco_objective_function(
        trial=trial,
        params=params,
        tuning_graphs=tuning_graphs,
        aco_class=ACOGraphColoring,
        verbose=ACO_VERBOSE,
        tabu_best_values=tabu_best_values,
        n_startup_trials=N_STARTUP_TRIALS,
    )

print("Objective function wrapper ready!")

Tuner initialized with study: aco_study_limited_dataset_20251130_195639
Data root: /meta/graph_coloring_antcol/assignemnt3/data

Loading tuning dataset...

Loading Tuning Dataset: limited_dataset
  gc_500_9:
    Nodes: 500, Edges: 112224
    Density: 0.8996, Connected: True

Graphs ordered by size (smallest to largest):
  1. gc_500_9: 500 nodes, 112224 edges


Loaded Tabu Tuning Best Results: 8 graphs
  gc_1000_7: 195 colors (from tabu search tuning)
  gc_100_9: 38 colors (from tabu search tuning)
  gc_20_9: 10 colors (from tabu search tuning)
  gc_250_9: 85 colors (from tabu search tuning)
  gc_4_1: 2 colors (from tabu search tuning)
  gc_500_9: 162 colors (from tabu search tuning)
  gc_50_9: 21 colors (from tabu search tuning)
  gc_70_9: 28 colors (from tabu search tuning)

Loaded 1 graphs for tuning

Objective function wrapper ready!


## 3. Run Hyperparmeters Tuning

In [7]:
# Run the hyperparameter optimization
print(f"\nStarting hyperparameter optimization with {N_TRIALS} trials...")
print("=" * 70)

best_params = tuner.optimize(
    objective_func=objective_wrapper,
    param_config=PARAM_CONFIG,
    aco_class=ACOGraphColoring,
    n_trials=N_TRIALS,
    n_jobs=5
)

print("\n" + "=" * 70)
print("Optimization completed!")
print("\nBest Parameters Found:")
for param_name, param_value in best_params.items():
    print(f"  {param_name}: {param_value}")

[I 2025-11-30 19:56:40,141] A new study created in Journal with name: aco_study_limited_dataset_20251130_195639



Starting hyperparameter optimization with 40 trials...
✓ Created new study 'aco_study_limited_dataset_20251130_195639'
  Storage: /meta/graph_coloring_antcol/assignemnt3/data/studies/aco_study_limited_dataset_20251130_195639/aco_study_limited_dataset_20251130_195639.log
  Random exploration trials: 4 (before optimization)
Trial Summary:
  Successful: 0/40
  Total executed: 0
  Remaining needed: 40

Starting optimization with 40 trials...


  0%|          | 0/40 [00:00<?, ?it/s]


Trial 1: Random Exploration (Startup 2/4)
Parameters:
  iterations: 324
  alpha: 0.7359189766547534
  beta: 4.918280272331653
  rho: 0.4777623114454349
  ant_count: 56
  Q: 1.1959287320058467
  patience: 0.6493768368239913
----------------------------------------------------------------------

💡 Trial 1 | Graph 'gc_500_9' | [1/1] | Starting (nodes=500, edges=112224, tabu_best=162)

Trial 0: Random Exploration (Startup 1/4)
Parameters:
  iterations: 414
  alpha: 0.912284245542913
  beta: 4.5176798421019795
  rho: 0.40921387387548347
  ant_count: 48
  Q: 3.737977758863831
  patience: 0.568828255871098
----------------------------------------------------------------------

💡 Trial 0 | Graph 'gc_500_9' | [1/1] | Starting (nodes=500, edges=112224, tabu_best=162)

Trial 2: Random Exploration (Startup 3/4)
Parameters:
  iterations: 392
  alpha: 1.075777565432095
  beta: 4.8532422337034244
  rho: 0.011516849729029409
  ant_count: 67
  Q: 8.681244803914215
  patience: 0.5350320274193499
------

In [8]:
# Generate all optimization plots
print("\nGenerating optimization visualization plots...")
print("=" * 70)
tuner.generate_plots(recreate=True)
print("=" * 70)


Generating optimization visualization plots...

Generating visualization plots...
Optimization history saved to: history.png
Parameter importances saved to: importances.png
Slice plot saved to: slice.png
Timeline plot saved to: timeline.png
All plots generated successfully.



### Best Trial Results

In [9]:
# Display best trial graphs and metrics


# Get best trial number
best_trial_number = tuner.study.best_trial.number
best_trial_dir = Path(DATA_ROOT) / 'studies' / STUDY_NAME / 'results' / f'trial_{best_trial_number:04d}'

print(f"Best Trial: {best_trial_number}")
print(f"Best Objective Value: {tuner.study.best_value}")
print("=" * 70)

# Display metrics for best trial
display(Markdown(f"#### Best Trial Metrics"))

metric_files = [
    ('color_count.png', 'Color Count per Graph'),
    ('execution_time.png', 'Execution Time per Graph'),
    ('conflicts.png', 'Conflicts per Graph')
]

for filename, title in metric_files:
    metric_path = best_trial_dir / filename
    if metric_path.exists():
        display(Markdown(f"**{title}**"))
        display(HTML(f'<img src="{metric_path}" style="width: 75%; display: block; margin: 0 auto;">'))

# Display colored graphs for best trial
display(Markdown(f"#### Best Trial: Colored Graph Solutions"))

# Find all graph files in best trial directory
graph_files = sorted(best_trial_dir.glob('graph_*.png'))
for graph_file in graph_files:
    graph_name = graph_file.stem.replace('graph_', '')
    display(Markdown(f"**{graph_name}**"))
    display(HTML(f'<img src="{graph_file}" style="width: 75%; display: block; margin: 0 auto;">'))

Best Trial: 19
Best Objective Value: 200.0


#### Best Trial Metrics

**Color Count per Graph**

**Execution Time per Graph**

**Conflicts per Graph**

#### Best Trial: Colored Graph Solutions

**gc_500_9**

### Display Study Visualizations

In [15]:
# Display all study figures

study_figures_path = Path(DATA_ROOT) / 'studies' / STUDY_NAME / 'figures'

# List of all possible study figures with descriptions
figure_files = [
    ('history.png', 'Optimization History', 'Shows objective value progression across trials'),
    ('importances.png', 'Parameter Importances', 'Shows which hyperparameters impact results most'),
    ('slice.png', 'Slice Plots (All Parameters)', 'Shows how each parameter affects objective value'),
    ('timeline.png', 'Trial Timeline', 'Shows when each trial ran and how long it took')
]

print("Study Visualization Figures:")
print("=" * 70)

for filename, title, description in figure_files:
    figure_path = study_figures_path / filename
    if figure_path.exists():
        display(Markdown(f"### {title}"))
        display(Markdown(f"*{description}*"))
        display(HTML(f'<img src="{figure_path}" style="width: 75%; display: block; margin: 0 auto;">'))
        print(f"✓ {filename}")
    else:
        print(f"✗ {filename} (not generated - may require additional dependencies)")

print("=" * 70)

Study Visualization Figures:


### Optimization History

*Shows objective value progression across trials*

✓ history.png


### Parameter Importances

...

*Shows how each parameter affects objective value*

✓ slice.png


### Trial Timeline

*Shows when each trial ran and how long it took*

✓ timeline.png


## 4. Comprehensive Algorithms Comparison

Compare three algorithms on the testing dataset:
- **Greedy Algorithm**: Fast constructive heuristic
- **Tabu Search**: Single-solution metaheuristic with memory
- **ACO**: Population-based metaheuristic (using best parameters from tuning)

Each algorithm runs 5 times on all test graphs to calculate statistics and generate comparison plots.

In [14]:
# Import the comprehensive testing module
from comprehensive_testing import run_comprehensive_testing

# Set up testing directories - save comparison inside study path
comparison_output_dir = DATA_ROOT / 'studies' / STUDY_NAME / 'algorithm_comparison'

print(f"Data root: {DATA_ROOT}")
print(f"Dataset: {DATASET_NAME}")
print(f"Comparison output directory: {comparison_output_dir}")
print(f"  (Results will be saved in study: {STUDY_NAME})")

Data root: /meta/graph_coloring_antcol/assignemnt3/data
Dataset: limited_dataset
Comparison output directory: /meta/graph_coloring_antcol/assignemnt3/data/studies/aco_study_limited_dataset_20251130_195639/algorithm_comparison
  (Results will be saved in study: aco_study_limited_dataset_20251130_195639)


In [16]:
# Prepare ACO parameters from best trial
aco_test_params = {
    'iterations': int(best_params['iterations']),
    'alpha': best_params['alpha'],
    'beta': best_params['beta'],
    'rho': best_params['rho'],
    'ant_count': int(best_params['ant_count']),
    'Q': best_params['Q'],
    'patience': best_params['patience']
}

print("\nACO Parameters for Testing (from best trial):")
for param, value in aco_test_params.items():
    print(f"  {param}: {value}")


ACO Parameters for Testing (from best trial):
  iterations: 261
  alpha: 1.5359605408264265
  beta: 5.965522373162997
  rho: 0.0972874079673251
  ant_count: 82
  Q: 1.2989056265137102
  patience: 0.5767930734112034


In [17]:
# Run comprehensive testing: 3 repetitions per algorithm per graph
# Results are cached in JSON - set force_rerun=True to re-execute tests
NUM_REPETITIONS = 3
FORCE_RERUN = False  # Set to True to ignore cached results and rerun testing

print(f"\n{'='*80}")
print(f"Running comprehensive testing with {NUM_REPETITIONS} repetitions per algorithm")
if FORCE_RERUN:
    print("⚠ FORCE_RERUN=True: Will ignore cached results and rerun all tests")
else:
    print("ℹ Using cached results if available (set FORCE_RERUN=True to rerun)")
print(f"{'='*80}\n")

df_comparison_results, df_comparison_stats = run_comprehensive_testing(
    data_root=str(DATA_ROOT),
    dataset_name=DATASET_NAME,
    output_dir=str(comparison_output_dir),
    num_repetitions=NUM_REPETITIONS,
    aco_params=aco_test_params,
    force_rerun=FORCE_RERUN
)

print("\n✓ Comprehensive testing complete!")


Running comprehensive testing with 3 repetitions per algorithm
ℹ Using cached results if available (set FORCE_RERUN=True to rerun)


Loaded Best Known Results: 3 graphs
  dsjc1000.5: 85 colors
  dsjc250.5: 28 colors
  dsjc500.9: 126 colors


Loading Testing Dataset: limited_dataset
  dsjc1000.5:
    Nodes: 1000, Edges: 249826
    Density: 0.5002, Connected: True
  dsjc250.5:
    Nodes: 250, Edges: 15668
    Density: 0.5034, Connected: True
  dsjc500.9:
    Nodes: 500, Edges: 112437
    Density: 0.9013, Connected: True

Graphs ordered by size (smallest to largest):
  1. dsjc250.5: 250 nodes, 15668 edges
  2. dsjc500.9: 500 nodes, 112437 edges
  3. dsjc1000.5: 1000 nodes, 249826 edges

Checking for existing test results...
COMPREHENSIVE ALGORITHM TESTING
Found 3 test graphs
Running 3 repetitions per graph per algorithm

Processing: dsjc250.5
  Nodes: 250, Edges: 15668
  Running Greedy...
    Rep 1: colors=42, conflicts=0, time=0.039s
    Rep 2: colors=42, conflicts=0, time=0.039s
    Re

### 6.1 Display Statistics Summary

In [18]:
# Display statistics table
from IPython.display import display, Markdown

print("Statistics Summary:")
display(df_comparison_stats)

# Print formatted statistics (without markdown)
print("\n" + "="*80)
print(df_comparison_stats.to_string(index=False))
print("="*80)

Statistics Summary:


,graph,best_known,algorithm,best_colors,avg_colors,std_colors,avg_conflicts,avg_time,std_time,deviation_best,deviation_avg
0,dsjc250.5,28,Greedy,42,42.000000,0.000000,0.0,0.037765,0.001972,50.000000,50.000000
1,dsjc250.5,28,Tabu Search,34,34.666667,0.577350,0.0,925.661106,241.689189,21.428571,23.809524
2,dsjc250.5,28,ACO,55,55.666667,0.577350,0.0,1134.350811,59.201912,96.428571,98.809524
3,dsjc500.9,126,Greedy,174,174.000000,0.000000,0.0,0.244075,0.017054,38.095238,38.095238
4,dsjc500.9,126,Tabu Search,164,166.000000,2.000000,0.0,6730.858449,2541.860045,30.158730,31.746032
5,dsjc500.9,126,ACO,199,201.333333,2.516611,0.0,5853.500459,42.272999,57.936508,59.788360
6,dsjc1000.5,85,Greedy,125,125.000000,0.000000,0.0,0.731298,0.048360,47.058824,47.058824
7,dsjc1000.5,85,Tabu Search,124,124.666667,0.577350,0.0,10026.232540,168.658511,45.882353,46.666667
8,dsjc1000.5,85,ACO,226,227.000000,1.000000,0.0,13669.638874,1333.957182,165.882353,167.058824



     graph  best_known   algorithm  best_colors  avg_colors  std_colors  avg_conflicts     avg_time    std_time  deviation_best  deviation_avg
 dsjc250.5          28      Greedy           42   42.000000    0.000000            0.0     0.037765    0.001972       50.000000      50.000000
 dsjc250.5          28 Tabu Search           34   34.666667    0.577350            0.0   925.661106  241.689189       21.428571      23.809524
 dsjc250.5          28         ACO           55   55.666667    0.577350            0.0  1134.350811   59.201912       96.428571      98.809524
 dsjc500.9         126      Greedy          174  174.000000    0.000000            0.0     0.244075    0.017054       38.095238      38.095238
 dsjc500.9         126 Tabu Search          164  166.000000    2.000000            0.0  6730.858449 2541.860045       30.158730      31.746032
 dsjc500.9         126         ACO          199  201.333333    2.516611            0.0  5853.500459   42.272999       57.936508      59.78836

### 6.2 Display Comparison Visualizations

In [19]:
# Display algorithm comparison plots (4 comparison metrics)
from IPython.display import Image

# Define all comparison plots
comparison_plots = [
    ('comparison_best_colors.png', 'Best Color Count Comparison (with Best Known Solutions)'),
    ('comparison_avg_colors.png', 'Average Color Count Comparison (with Std Dev)'),
    ('comparison_execution_time.png', 'Execution Time Comparison (log scale)'),
    ('comparison_conflicts.png', 'Conflict Count Comparison')
]

print("Algorithm Comparison Metrics:")
print("="*80)

for filename, title in comparison_plots:
    plot_path = comparison_output_dir / filename
    if plot_path.exists():
        display(Markdown(f"### {title}"))
        display(Markdown(f'<img src="{plot_path}" style="width: 75%; display: block; margin: 0 auto;"/>'))
    else:
        print(f"⚠ {title} not found: {filename}")

print("="*80)

Algorithm Comparison Metrics:


### Best Color Count Comparison (with Best Known Solutions)

<img src="/meta/graph_coloring_antcol/assignemnt3/data/studies/aco_study_limited_dataset_20251130_195639/algorithm_comparison/comparison_best_colors.png" style="width: 75%; display: block; margin: 0 auto;"/>

### Average Color Count Comparison (with Std Dev)

<img src="/meta/graph_coloring_antcol/assignemnt3/data/studies/aco_study_limited_dataset_20251130_195639/algorithm_comparison/comparison_avg_colors.png" style="width: 75%; display: block; margin: 0 auto;"/>

### Execution Time Comparison (log scale)

<img src="/meta/graph_coloring_antcol/assignemnt3/data/studies/aco_study_limited_dataset_20251130_195639/algorithm_comparison/comparison_execution_time.png" style="width: 75%; display: block; margin: 0 auto;"/>

### Conflict Count Comparison

<img src="/meta/graph_coloring_antcol/assignemnt3/data/studies/aco_study_limited_dataset_20251130_195639/algorithm_comparison/comparison_conflicts.png" style="width: 75%; display: block; margin: 0 auto;"/>

### 6.3 Results Summary and File Locations

In [20]:
# Print comprehensive testing results summary
print("="*80)
print("COMPREHENSIVE TESTING RESULTS SUMMARY")
print("="*80)

print(f"\n📁 Output Directory: {comparison_output_dir}")
print(f"   (Saved in study: {STUDY_NAME})")

print(f"\n📊 Generated Files:")
print(f"   - comparison_results.json (complete results - cached for reuse)")
print(f"   - comprehensive_test_results.csv (all {len(df_comparison_results)} individual runs)")
print(f"   - statistics_summary.csv (aggregated statistics)")
print(f"   - comparison_best_colors.png (best color count comparison)")
print(f"   - comparison_avg_colors.png (average color count with std dev)")
print(f"   - comparison_execution_time.png (execution time comparison)")
print(f"   - comparison_conflicts.png (conflict count comparison)")
print(f"   - color_distribution_boxplots.png (distribution analysis)")

print(f"\n💡 Tip: Results are cached in JSON. Re-running this cell will use cached data.")
print(f"   Set FORCE_RERUN=True to ignore cache and rerun all tests.")

print(f"\n🎯 Summary by Algorithm:")
for algo_name in df_comparison_stats['algorithm'].unique():
    df_algo = df_comparison_stats[df_comparison_stats['algorithm'] == algo_name]
    best_colors = df_algo['best_colors'].min()
    avg_colors = df_algo['avg_colors'].mean()
    avg_time = df_algo['avg_time'].mean()
    print(f"   {algo_name}:")
    print(f"      Best colors: {best_colors}")
    print(f"      Avg colors: {avg_colors:.2f}")
    print(f"      Avg time: {avg_time:.4f}s")

print("\n" + "="*80)

COMPREHENSIVE TESTING RESULTS SUMMARY

📁 Output Directory: /meta/graph_coloring_antcol/assignemnt3/data/studies/aco_study_limited_dataset_20251130_195639/algorithm_comparison
   (Saved in study: aco_study_limited_dataset_20251130_195639)

📊 Generated Files:
   - comparison_results.json (complete results - cached for reuse)
   - comprehensive_test_results.csv (all 27 individual runs)
   - statistics_summary.csv (aggregated statistics)
   - comparison_best_colors.png (best color count comparison)
   - comparison_avg_colors.png (average color count with std dev)
   - comparison_execution_time.png (execution time comparison)
   - comparison_conflicts.png (conflict count comparison)
   - color_distribution_boxplots.png (distribution analysis)

💡 Tip: Results are cached in JSON. Re-running this cell will use cached data.
   Set FORCE_RERUN=True to ignore cache and rerun all tests.

🎯 Summary by Algorithm:
   Greedy:
      Best colors: 42
      Avg colors: 113.67
      Avg time: 0.3377s
   Ta